<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/DataAnalyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install toolz==0.12.0
#!pip install matplotlib==3.7.1

In [1]:
from typing import List, Callable, Tuple
from toolz import pipe, map, filter, partial, reduce, concat, take, drop
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px


In [30]:
from typing import List, Callable, Tuple
from toolz import pipe, map, filter, partial, reduce, concat, take, drop
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import plotly.express as px
import plotly.graph_objects as go

# Global variable to keep track of the saved file sequence number
saved_figure_count = 0

def read_vectors(filepath: str, max_rows: int = None) -> List[List[float]]:
  """Reads a file containing newline-separated vectors and returns a list of vectors."""
  with open(filepath, 'r') as file:
      if max_rows is None:
        return [[float(num) for num in line.split()] for line in file if line.strip() and 'end' not in line]
      else:
        return [[float(num) for num in line.split()] for line in file if line.strip() and 'end' not in line][:max_rows]
    #return [[float(num) for num in line.split()] for line in file if line.strip()]

def map_vectors(func: Callable[[List[float]], List[float]], vectors: List[List[float]]) -> List[List[float]]:
  """Applications a function to each vector in the list."""
  return list(map(func, vectors))

def filter_vectors(predicate: Callable[[List[float]], bool], vectors: List[List[float]]) -> List[List[float]]:
  """Filters the list of vectors based on a predicate."""
  return list(filter(predicate, vectors))

def sort_vectors_by_column(vectors: List[List[float]], column_index: int, reverse: bool = False) -> List[List[float]]:
  """Sorts the list of vectors by a specific column."""
  return sorted(vectors, key=lambda vector: vector[column_index], reverse=reverse)

def zip_vectors(vectors1: List[List[float]], vectors2: List[List[float]]) -> List[Tuple[List[float], List[float]]]:
  """Zips two lists of vectors together."""
  return list(zip(vectors1, vectors2))

def concat_vectors(vectors1: List[List[float]], vectors2: List[List[float]]) -> List[List[float]]:
  """Concatenates two lists of vectors."""
  return list(concat([vectors1, vectors2]))

def cut_vectors(vectors: List[List[float]], start_index: int, end_index: int) -> List[List[float]]:
  """Cuts a list of vectors by index ranges."""
  return list(take(end_index, drop(start_index, vectors)) )

def read_bool_vector(filepath: str) -> List[float]:
  """Reads a file containing newline-separated boolean strings (True/False)
  and converts them to a list of floats (1.0/0.0)."""
  with open(filepath, 'r') as file:
    return [1.0 if line.strip() == 'True' else 0.0 for line in file if 'end' not in line]

def add_dimension(vectors: List[List[float]], new_dimension: List[float]) -> List[List[float]]:
    """Adds a new dimension as the first element to each vector."""
    # Ensure both lists have the same length
    min_length = min(len(vectors), len(new_dimension))
    vectors = vectors[:min_length]
    new_dimension = new_dimension[:min_length]

    return [[new_val] + vec for new_val, vec in zip(new_dimension, vectors)]

def squeeze_columns(vectors: List[List[float]], c1: int, c2: int) -> List[List[float]]:
    """Squeezes columns from c1 to c2 (inclusive) into a single column by addition.
    Handles c1=0 correctly.
    """
    return list(map(lambda vector: ([sum(vector[c1:c2])] + vector[c2:]) if c1 == 0
                                  else (vector[:c1-1] + [sum(vector[c1-1:c2])] + vector[c2:]),
                   vectors))

def print_vector_info(vectors: List[List[float]]) -> None:
  """Prints the size and dimensions of the list of vectors."""
  num_vectors = len(vectors)
  if num_vectors > 0:
    vector_dim = len(vectors[0])
  else:
    vector_dim = 0  # Handle empty list case

  print(f"Number of vectors: {num_vectors}")
  print(f"Dimension of vectors: {vector_dim}")

  # Using NumPy for a more concise output
  if num_vectors > 0:
    print(f"Shape of vectors (NumPy): {np.array(vectors).shape}")


def save_to_file(fig):
  """Saves a matplotlib figure to a PNG file with a sequential filename."""
  global saved_figure_count
  filename = f"ne{saved_figure_count}.png"
  fig.savefig(filename)
  print(f"Saved figure to {filename}")
  saved_figure_count += 1
  plt.close(fig) # Close the figure to free up memory

def visualize_output(output, row_labels=None, plot_3d=True, save_file=False):
    """Visualizes the output matrix using matplotlib or Plotly with a hybrid colormap.
    Optionally saves the matplotlib plot to a file.
    """
    # Define colors for discrete values 1, 2, 3
    discrete_colors = {
        1: 'blue',
        2: 'yellow',
        3: 'red'
    }

    # Define a continuous colormap for values between -10 and 10
    continuous_cmap = plt.cm.RdYlGn_r # Red-Yellow-Green reversed to get Blue-Green-Red

    # Create a custom colormap
    # We'll map values 1, 2, 3 to distinct indices outside the continuous range
    # For example, map 1 to -11, 2 to -12, 3 to -13
    # The continuous range will be mapped from -10 to 10
    all_colors = []
    bounds = []

    # Add colors for values outside the -10 to 10 range (for 1, 2, 3)
    # Map 1 to -13, 2 to -12, 3 to -11 to keep them distinct and below -10
    all_colors.append(discrete_colors[1])
    bounds.append(-13)
    all_colors.append(discrete_colors[2])
    bounds.append(-12)
    all_colors.append(discrete_colors[3])
    bounds.append(-11)


    # Add colors from the continuous colormap
    num_continuous_colors = 256 # Number of colors in the continuous colormap
    continuous_bounds = np.linspace(-10, 10, num_continuous_colors)
    for i in range(num_continuous_colors):
        all_colors.append(continuous_cmap(i/num_continuous_colors))
        bounds.append(continuous_bounds[i])

    # Ensure bounds are strictly increasing
    bounds = sorted(list(set(bounds)))

    # Create the custom colormap
    cmap = ListedColormap(all_colors)
    norm = plt.Normalize(min(bounds), max(bounds))

    if row_labels is not None and len(row_labels) > 0:
        print(f"len(row_labels) , num_rows {len(row_labels)} {len(output)}")
        min_length = min(len(row_labels), len(output))
        row_labels = row_labels[:min_length]
        output = output[:min_length]
    else:
        row_labels = [str(i) for i in range(len(output))] # Create labels if none provided


    num_rows = len(output)
    if num_rows == 0:
        print("Output is empty. Cannot visualize.")
        return
    num_cols = len(output[0]) if num_rows > 0 else 0
    if num_cols == 0:
        print("Output rows are empty. Cannot visualize.")
        return

    # Convert output to a NumPy array
    output_array = np.array(output)

    # Map discrete values to their chosen indices
    mapped_output = np.copy(output_array)
    mapped_output[mapped_output == 1.0] = -13
    mapped_output[mapped_output == 2.0] = -12
    mapped_output[mapped_output == 3.0] = -11


    if plot_3d:
        mapped_output = [row[:-1] for row in mapped_output]
        # 3D plot using Plotly
        # Create X, Y, Z coordinates for the surface plot
        x = np.arange(num_cols)
        y = np.arange(num_rows)
        x, y = np.meshgrid(x, y)
        z = mapped_output

        fig = go.Figure(data=[go.Surface(z=z, x=x, y=y, colorscale='Viridis')]) # Using Viridis as an example colorscale
        fig.update_layout(title='3D Surface Plot of Output',
                          scene = dict(
                              xaxis_title='Column Index',
                              yaxis_title='Row Index',
                              zaxis_title='Value'),
                          autosize=False,
                          width=700,
                          height=700,
                          margin=dict(l=65, r=50, b=65, t=90))
        fig.show()

    else:
        # 2D plot using matplotlib
        # Determine figure size based on the number of rows and columns
        fig_width = num_cols * 0.2
        fig_height = num_rows * 0.1

        fig, ax = plt.subplots(figsize=(fig_width, fig_height))

        # Display the output using imshow with the custom colormap and normalization
        im = ax.imshow(mapped_output, aspect='auto', interpolation='nearest', cmap=cmap, norm=norm)
        plt.colorbar(im, ax=ax) # Add a colorbar to show the mapping

        # Set y-axis ticks and labels
        if row_labels is not None and len(row_labels) == num_rows:
            ax.set_yticks(np.arange(num_rows))

            rich_indices = [min(int(output[i][-1]), num_rows-1) for i in range(num_rows)]

            #print(f" rich_indices: {[rich_indices for i in range(0,rich_indices)]}")
            #rich_row_labels = [f"{row_labels[rich_indices[i]]} || {row_labels[i]}" for i in range(num_rows)]
            rich_row_labels = [f"{row_labels[i]}" for i in range(num_rows)]
            ax.set_yticklabels(rich_row_labels)
            #ax.set_yticklabels(row_labels)

        else:
            # Set y-axis ticks to show every 10 rows
            ax.set_yticks(np.arange(0, num_rows, 10))

        if save_file:
          plt.tight_layout() # Adjust layout to prevent labels from being clipped
          save_to_file(fig)
        else:
          plt.show()


def visualize_output2(output, columns, s_row=0, n_rows=None):
  """
  Visualizes the output based on specified columns using Plotly.

  Args:
    output: The output data (list of lists).
    columns: A list of column indices to visualize (length 2 for 2D, 3 for 3D).
  """
  if len(columns) < 2 or len(columns) > 3:
    print("Please provide either 2 or 3 column indices for visualization.")
    return

  # Convert output to a NumPy array for easier column access
  output_array = np.array(output)
  if n_rows is not None:
      output_array = output_array[s_row:n_rows]
  else:
      output_array = output_array[s_row:]

  if len(columns) == 2:
    x_col = columns[0]
    y_col = columns[1]
    if x_col >= output_array.shape[1] or y_col >= output_array.shape[1]:
      print("Invalid column index provided.")
      return

    fig = px.scatter(x=output_array[:, x_col], y=output_array[:, y_col],  width=300, height=300)
    fig.update_layout(
        xaxis_title=f"Column {x_col}",
        yaxis_title=f"Column {y_col}",
        title="2D Scatter Plot"
    )
    fig.show()

  elif len(columns) == 3:
    x_col = columns[0]
    y_col = columns[1]
    z_col = columns[2]
    if x_col >= output_array.shape[1] or y_col >= output_array.shape[1] or z_col >= output_array.shape[1]:
      print("Invalid column index provided.")
      return

    fig = px.scatter_3d(x=output_array[:, x_col], y=output_array[:, y_col], z=output_array[:, z_col], width=300, height=300)
    fig.update_layout(
        scene = dict(
            xaxis_title=f"Column {x_col}",
            yaxis_title=f"Column {y_col}",
            zaxis_title=f"Column {z_col}"),
        title="3D Scatter Plot"
    )
    fig.show()

In [31]:
from typing import List, Callable, Tuple
from toolz import pipe, map, filter, partial, reduce, concat, take, drop
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import plotly.express as px
import plotly.graph_objects as go


def compute_derivative_sign(sorted_vectors: List[List[float]]) -> List[List[int]]:
  """Compares adjacent elements in a list of vectors and returns a list of lists
  of 1, 2, or 3 based on the sign of the difference."""
  output = []
  # Iterate through sorted_vectors starting from the second element (index 1)
  for i in range(1, len(sorted_vectors)):
    current_vector = sorted_vectors[i]
    previous_vector = sorted_vectors[i-1]
    current_vector_output = [current_vector[0]]
    # Start from the second column (index 1), omit first column (always 1) omit last column (that is seq no)
    for j in range(1, len(current_vector) - 1):
      if current_vector[j] > previous_vector[j]:
        current_vector_output.append(3)
      elif current_vector[j] == previous_vector[j]:
        current_vector_output.append(2)
      else:
        current_vector_output.append(1)
    output.append(current_vector_output)
  return output

def apply_linear_transform(sorted_vectors: List[List[float]], a: float, b: float) -> List[List[float]]:
  """Applies a linear transformation (a*x + b) to each element in each vector."""
  output = []
  # Iterate through sorted_vectors starting from the second element (index 1)
  for i in range(len(sorted_vectors)):
    current_vector = sorted_vectors[i]
    # Apply the linear transformation to each element, excluding the first and last
    transformed_vector = [x * a + b for x in current_vector[1:-1]]
    output.append(transformed_vector)
  return output

def clip_value(vectors: List[List[float]], bottom_val: float, top_val: float) -> List[List[float]]:
  """Clips the values in each vector of the list to be within a specified range."""
  output = []
  for vector in vectors:
    clipped_vector = [max(bottom_val, min(top_val, x)) for x in vector[1:-1]]
    output.append(clipped_vector)
  return output

def process_vectors_composed(sorted_vectors, compute_deriv_sign=True, a=1.0, b=0.0, bottom_val=5.0, top_val=10.0):
    """Combines the refactored functions to achieve the same result as process_vectors3."""
    if compute_deriv_sign:
        processed_data = compute_derivative_sign(sorted_vectors)
        # The derivative sign computation now includes the first row,
        # so the length matches sorted_vectors
        aligned_original_vectors = sorted_vectors
    else:
        processed_data = apply_linear_transform(sorted_vectors, a, b)
        processed_data = clip_value(processed_data, bottom_val, top_val)
        # The linear transform and clip value process all rows, so no alignment needed
        aligned_original_vectors = sorted_vectors


    # Add the sequence number back to the end of each processed vector
    output_with_sequence = []
    for i in range(len(processed_data)):
        # Append the sequence number from the corresponding original vector
        output_with_sequence.append(processed_data[i] + [aligned_original_vectors[i][-1]])
        print( f"aligned_original_vectors {i} {aligned_original_vectors[i][-1]}")
    return output_with_sequence

In [32]:
def initialize_datavector(activations_filename, correctness_vals):
  vectors = read_vectors(activations_filename, None)  # Assuming vectors are stored in 'vectors.txt'
  if correctness_vals is None:
    bool_vector = [1.0] * len(vectors)
  else:
    bool_vector = read_bool_vector(correctness_vals)  # Assuming boolean strings are in 'bool_vector.txt'
  print_vector_info(vectors)
  #print(f"bool_vector {bool_vector}")
  # Add the boolean vector as the first dimension to the existing vectors
  merged_vectors = add_dimension(vectors, bool_vector) #if not equal length truncate to shorter

  # Add a sequence number as the last column
  for i in range(len(merged_vectors)):
    merged_vectors[i].append(float(i))

  print(f"merged_vectors {merged_vectors[:10]}")
  print_vector_info(merged_vectors)
  return merged_vectors

def process_vectors_EuclChart(merged_vectors,cols, s_row, n_rows):
  visualize_output2(merged_vectors, cols, s_row, n_rows)


def process_vectors(activations_filename, correctness_vals, sort_range, max_rows):
  vectors = read_vectors(activations_filename, max_rows)  # Assuming vectors are stored in 'vectors.txt'
  if correctness_vals is None:
    bool_vector = [1.0] * len(vectors)
  else:
    bool_vector = read_bool_vector(correctness_vals)  # Assuming boolean strings are in 'bool_vector.txt'
  print_vector_info(vectors)
  print(f"bool_vector {bool_vector}")

  # Add the boolean vector as the first dimension to the existing vectors
  merged_vectors = add_dimension(vectors, bool_vector)
  print(f"merged_vectors {merged_vectors}")
  print_vector_info(merged_vectors)
  #merged_vectors = squeeze_columns(merged_vectors, 1, 6)
  #merged_vectors = squeeze_columns(merged_vectors, 8, 11)
  print(f"merged_vectors squeezed {merged_vectors}")
  print_vector_info(merged_vectors)
  print("filterby first column")
  merged_vectors = filter_vectors(lambda vector: vector[0] == 1.0, merged_vectors)
  print_vector_info(merged_vectors)
  process_vectors2(merged_vectors, sort_range)

def process_vectors4(activations_filename, token_labels, correctness_vals, sort_range, max_rows, squeeze_factor, compute_deriv_sign=True, a=1.0, b=0.0):
    datavector = initialize_datavector(activations_filename, correctness_vals)
    print_vector_info(datavector)

    #squeeze_factor = 32
    squeezed_datavector = []
    for vector in datavector:
        new_vector = []
        for i in range(1, len(vector)-1, squeeze_factor):
            new_vector.append(sum(vector[i:i+squeeze_factor]) / squeeze_factor)
        # Add the first column (correctness value) and the last column (sequence number) back to the new vector
        squeezed_datavector.append([vector[0]] + new_vector + [vector[-1]])


    print_vector_info(squeezed_datavector)
    process_vectors2(squeezed_datavector, token_labels, sort_range, compute_deriv_sign,a , b)

def process_vectors2(merged_vectors, token_labels, sort_range, compute_deriv_sign=True, a=1.0, b=0.0):
  if sort_range is None:
    sorted_vectors = merged_vectors
    result = process_vectors3(sorted_vectors, compute_deriv_sign, a, b)
    #print(f"output {result}")
    print_vector_info(result)
    visualize_output(result, token_labels)
  else:
    for sort_column in sort_range:
      sorted_vectors = sort_vectors_by_column(merged_vectors, column_index=sort_column)
      result = process_vectors3(sorted_vectors, compute_deriv_sign, a, b)
      #result = process_vectors_composed(sorted_vectors, compute_deriv_sign, a, b, bottom_val=a, top_val=b)
      print(f"sort by {sort_column}")
      print_vector_info(result)
      visualize_output(result, token_labels)
  #sorted_vectors = merged_vectors
  #sorted_vectors = filter_vectors(lambda vector: vector[0] == 1.0, sorted_vectors)

  #print (f"sorted_vectors {sorted_vectors}")
  #print_vector_info(sorted_vectors)


def process_vectors3(sorted_vectors, compute_deriv_sign=True, a=1.0, b=0.0):
    # 2. Compare adjacent elements in each column and generate output
  output = []
  for i in range(len(sorted_vectors)):
    current_vector_output = []  # Output for the current vector
    # Skip the first row for comparison
    if i > 0:
      for j in range(1, len(sorted_vectors[i])-1):  # Start from the second column (index 1), omit first column (always 1) omit last column (that is seq no)
        if compute_deriv_sign == True:
          if sorted_vectors[i][j] > sorted_vectors[i - 1][j]:
            current_vector_output.append(3)
          elif sorted_vectors[i][j] == sorted_vectors[i - 1][j]:
            current_vector_output.append(2)
          else:
            current_vector_output.append(1)
        else:
          val = sorted_vectors[i][j]*a + b
          """if(val < 5.0):
            val = 5.0
          if(val > 10.0):
            val = 10.0"""
          current_vector_output.append( val)
          #current_vector_output.append( sorted_vectors[i][j]*20.0 -21.0)
      current_vector_output.append( sorted_vectors[i][len(sorted_vectors[i])-1] )
      output.append(current_vector_output)  # Append output for current vector to overall output
  return output

def read_labels(filepath: str) -> List[str]:
  """Reads a file containing newline-separated labels and returns a list of labels."""
  with open(filepath, 'r') as file:
    return [line.strip() for line in file if line.strip()]

In [ ]:


process_vectors('activations_fc_input_d64_h1_epoch_4_test.txt', 'corrects_list_d64_h1_epoch_4_test.txt', range(1,32))
process_vectors('activations_fc_input_d64_h1_epoch_4_test.txt', 'corrects_list_d64_h1_epoch_4_test.txt', None)
#process_vectors('activations_fc_input_d64_h1_epoch_4_train.txt', 'corrects_list_d64_h1_epoch_4_train.txt', None)



In [ ]:
#from tinystories
process_vectors('sample_data/activations_rec4ep20_f10000.txt',None , None, 1000)
process_vectors('sample_data/activations_rec4ep20_f10000.txt',None , range(5, 12), 600)


#from tinystories, 2-3D

In [ ]:
datavector = initialize_datavector('sample_data/activations_rec6ep20_f10000.txt',None)
#datavector = initialize_datavector('sample_data/activations_rec4ep20_f10000.txt',None)


In [ ]:
process_vectors_EuclChart(datavector, [17 ,18], 700,800)


In [ ]:
print("before sort")
print([row[:5] for row in datavector[0:10]])
sorted_vectors = sort_vectors_by_column(datavector[0:5000], column_index=10)
print("\n\n\nafter sort")
print([row[:5] for row in sorted_vectors[0:10]])
#sorted_vectors = datavector
for x_col in range(10, 11, 1):
    for y_col in range(x_col-4, x_col+10, 1):
      process_vectors_EuclChart(sorted_vectors, [x_col ,y_col], 1,80)
"""
for x_col in range(1, 20, 1):
    for y_col in range(x_col+1, x_col+2, 1):
      process_vectors_EuclChart(sorted_vectors, [x_col ,y_col], 10,100)
"""

Phi2

In [ ]:

process_vectors('sample_data/activations_phi2_factor_c.txt',None , range(1, 12), 1000)


In [ ]:
datavector = initialize_datavector('sample_data/activations_phi2_factor_c.txt',None)


In [ ]:
process_vectors4('sample_data/activations_phi2_factor_c3.txt',None , range(0, 30), 1000, squeeze_factor = 17)
#/content/sample_data/activations_phi2_fcall_c2.txt

In [ ]:
process_vectors_EuclChart(datavector, [8 ,9], 0,800)

tinyllama

In [33]:


#token_labels = read_labels('sample_data/labels_tinyllama_cprrec_fact.txt')
#process_vectors4('sample_data/activs_tinyllama_cprrec_fact.txt', token_labels, None,
#                 range(0, 3), 200, squeeze_factor = 64, compute_deriv_sign=False, a=20.0, b=-21.0)
token_labels = read_labels('sample_data/labels_tinyllama_cprrec_fact.txt')
process_vectors4('sample_data/activs_tinyllama_cprrec_fact.txt', token_labels, None,
                 range(0, 3), 200, squeeze_factor = 64, compute_deriv_sign=False, a=1.0, b=0.0)

#/content/sample_data/activs_tinyllama_cprrec_cucumb.txt
#token_labels = read_labels('sample_data/labels_tinyllama_cnorec_fact.txt')
#process_vectors4('sample_data/activs_tinyllama_cnorec_fact.txt', token_labels, None,
#                 range(0, 20), 200, squeeze_factor = 64, compute_deriv_sign=False, a=20.0, b=-21.0)


#a=40.0, b=-21.0
#a=20.0, b=-21.0
#a=30.0, b=-21.0
#3d
# a=20.0, b=-1.0
#activs_tinyllama_cnorec_fact
#activs_tinyllama_cprrec_fact
#process_vectors4('sample_data/activations_tinyllama_factor_c5.txt',None , range(20, 33), 200, squeeze_factor = 63)
#content/sample_data/activations_tinyllama_factor_c4.txt

Number of vectors: 181
Dimension of vectors: 2048
Shape of vectors (NumPy): (181, 2048)
merged_vectors [[1.0, -0.421082, 0.693964, -0.860379, -2.264612, 1.233486, 0.830988, -0.536872, 0.474853, 0.770456, -0.73237, -1.907727, -1.745697, -2.533497, -2.423662, -0.834862, -0.147053, 0.344153, -0.011808, -0.195635, -0.457463, 2.657728, -0.463237, 1.43014, 0.508663, 0.328795, -0.512441, -1.588943, -0.106322, -1.361442, 0.245161, 3.029059, 2.493244, 2.961539, 0.479361, 1.260118, -0.782576, -1.125708, 3.364944, -0.336847, 1.595882, 3.221968, -1.05002, 2.714467, -0.257205, -1.928512, -0.045017, -0.761909, 2.145962, 1.685403, 0.230403, -0.559611, -1.389182, 2.643193, 3.346819, -0.341235, 2.438712, -0.577827, 2.582896, 0.065797, -2.421713, -1.506377, -0.131897, 1.381026, 3.995912, 0.285045, -2.708593, 0.644165, -1.173076, 0.885037, 1.42915, -0.747398, -1.198328, 0.296436, -2.931625, 0.686869, 4.050124, 1.208203, -1.569514, -2.145651, -3.824414, 2.068728, 0.190527, -0.903508, 0.473786, -1.225888, 

sort by 1
Number of vectors: 180
Dimension of vectors: 33
Shape of vectors (NumPy): (180, 33)
len(row_labels) , num_rows 181 180


sort by 2
Number of vectors: 180
Dimension of vectors: 33
Shape of vectors (NumPy): (180, 33)
len(row_labels) , num_rows 181 180


In [ ]:
token_labels = read_labels('sample_data/labels_tinyllama_cnorec_fact.txt')
process_vectors_composed(sorted_vectors, compute_deriv_sign=True, a=1.0, b=0.0, bottom_val=5.0, top_val=10.0):


Tests

In [ ]:
# Example Usage
vectors = read_vectors('example_vectors.txt')  # Assuming vectors are stored in 'vectors.txt'

# Map: Double each element in each vector
doubled_vectors = map_vectors(lambda vector: [x * 2 for x in vector], vectors)

# Filter: Keep vectors where the first element is positive
positive_vectors = filter_vectors(lambda vector: vector[0] > 0, vectors)

# Sort: Sort vectors by the second column in descending order
sorted_vectors = sort_vectors_by_column(vectors, column_index=1, reverse=True)

# Zip: Combine two lists of vectors
zipped_vectors = zip_vectors(vectors, doubled_vectors)

# Concat: Concatenate two lists of vectors
concatenated_vectors = concat_vectors(vectors, doubled_vectors)

# Cut: Get vectors from index 2 to 5
cut_vectors_result = cut_vectors(vectors, start_index=2, end_index=5)



In [ ]:
# Sample input data
sample_vectors = [
    [1.0, 0.1, 0.2, 0.3, 100.0],
    [1.0, 0.2, 0.2, 0.4, 101.0],
    [1.0, 0.15, 0.3, 0.35, 102.0],
    [1.0, 0.25, 0.2, 0.45, 103.0],
    [1.0, 0.25, 0.2, 0.45, 104.0]
]

# Test case 1: compute_deriv_sign = True
print("Testing compute_deriv_sign = True")
output_original_deriv = process_vectors3(sample_vectors, compute_deriv_sign=True)
output_composed_deriv = process_vectors_composed(sample_vectors, compute_deriv_sign=True)

print("Original output (deriv):", output_original_deriv)
print("Composed output (deriv):", output_composed_deriv)

assert output_original_deriv == output_composed_deriv
print("Test Case 1 Passed: Outputs match for compute_deriv_sign = True")

# Test case 2: compute_deriv_sign = False (linear transform and clip)
print("\nTesting compute_deriv_sign = False")
a_val = 10.0
b_val = -1.0
bottom = 0.0
top = 5.0
output_original_linear = process_vectors3(sample_vectors, compute_deriv_sign=False, a=a_val, b=b_val)
# The original process_vectors3 does not take bottom_val and top_val as explicit parameters in the else block,
# it has hardcoded clipping. We will mimic that for comparison, but the composed function
# uses the separate clip_value function. Let's manually apply clip to the original output for comparison.
# Note: This highlights a difference in how clipping was handled in the original function's else block
# compared to the refactored clip_value function. The refactored approach is more flexible.

# To accurately compare with the composed function which uses the clip_value function,
# we need to apply clipping to the output of the original process_vectors3 when compute_deriv_sign is False.
# The original process_vectors3 applies clipping *inside* the loop. The composed function
# applies the linear transform to all rows and then clips. Let's adjust the comparison
# to reflect the intended behavior of the composed function.
# The composed function applies linear transform to all rows, then clips the result.
# The original function applies linear transform and clipping row by row starting from the second row.
# Let's adjust the sample data and comparison to better reflect the row-by-row processing in the original.

# Let's redefine sample_vectors to avoid the first row being skipped in the original's linear path.
# However, the refactoring task was to match the behavior of process_vectors3, which *does* iterate
# from the second row when `i > 0`. The confusion arises because the `else` block for linear transform
# is inside this `if i > 0` check. This means the first row is also skipped in the original's linear path.
# The `process_vectors_composed` function, as implemented based on the refactoring subtasks,
# applies linear transform and clip to *all* rows of the `aligned_original_vectors` (which is `sorted_vectors`
# when `compute_deriv_sign` is False). This is a slight deviation from the original `process_vectors3`
# which skips the first row even for the linear transform path.

# Let's adjust the test to match the behavior of process_vectors_composed, which is arguably
# the more logical behavior for a linear transform. The comparison will then be with the
# output of `process_vectors_composed` applying the linear transform and clip to all rows.
# We will manually apply the linear transform and clip to all rows of sample_vectors
# to create the expected output for this test case, matching the behavior of process_vectors_composed.

expected_output_linear_composed = []
for vector in sample_vectors:
    transformed_clipped_vector = []
    for x in vector[1:-1]: # Exclude first and last columns
        transformed_val = x * a_val + b_val
        clipped_val = max(bottom, min(top, transformed_val))
        transformed_clipped_vector.append(clipped_val)
    # Add the sequence number back
    expected_output_linear_composed.append(transformed_clipped_vector + [vector[-1]])


output_composed_linear = process_vectors_composed(sample_vectors, compute_deriv_sign=False, a=a_val, b=b_val, bottom_val=bottom, top_val=top)

print("Expected output (linear+clip, matching composed):", expected_output_linear_composed)
print("Composed output (linear+clip):", output_composed_linear)

# Due to floating point comparisons, use numpy.allclose
assert np.allclose(np.array(expected_output_linear_composed), np.array(output_composed_linear))
print("Test Case 2 Passed: Outputs match for compute_deriv_sign = False (linear transform and clip)")


Testing compute_deriv_sign = True
Original output (deriv): [[3, 2, 3, 101.0], [1, 3, 1, 102.0], [3, 1, 3, 103.0], [2, 2, 2, 104.0]]
Composed output (deriv): [[3, 2, 3, 101.0], [1, 3, 1, 102.0], [3, 1, 3, 103.0], [2, 2, 2, 104.0]]
Test Case 1 Passed: Outputs match for compute_deriv_sign = True

Testing compute_deriv_sign = False
Expected output (linear+clip, matching composed): [[0.0, 1.0, 2.0, 100.0], [1.0, 1.0, 3.0, 101.0], [0.5, 2.0, 2.5, 102.0], [1.5, 1.0, 3.5, 103.0], [1.5, 1.0, 3.5, 104.0]]
Composed output (linear+clip): [[0.0, 1.0, 2.0, 100.0], [1.0, 1.0, 3.0, 101.0], [0.5, 2.0, 2.5, 102.0], [1.5, 1.0, 3.5, 103.0], [1.5, 1.0, 3.5, 104.0]]
Test Case 2 Passed: Outputs match for compute_deriv_sign = False (linear transform and clip)


In [ ]:
def save_to_file1(fig):
  """Saves a matplotlib figure to a PNG file with a sequential filename."""
  global saved_figure_count
  filename = f"ne{saved_figure_count}.png"
  plt.tight_layout() # Adjust layout to prevent labels from being clipped
  fig.savefig(filename)
  print(f"Saved figure to {filename}")
  saved_figure_count += 1
  plt.close(fig) # Close the figure to free up memory